# Week 3

In [1]:
! docker ps

CONTAINER ID   IMAGE                                           COMMAND                  CREATED              STATUS                        PORTS                                                                NAMES
9a38a3945315   grafana/grafana:9.4.7                           "/run.sh"                About a minute ago   Up About a minute             0.0.0.0:3000->3000/tcp                                               grafana
29680e882146   prom/prometheus:v2.43.0                         "/bin/prometheus --c…"   About a minute ago   Up About a minute             0.0.0.0:9090->9090/tcp                                               prometheus
2b444bf43c63   gcr.io/cadvisor/cadvisor:v0.47.1                "/usr/bin/cadvisor -…"   About a minute ago   Up About a minute (healthy)   0.0.0.0:8080->8080/tcp                                               cadvisor
cd28a23ad1fc   opensearchproject/opensearch:2.6.0              "./opensearch-docker…"   4 hours ago          Up 5 minutes             

#### Q1: Which node was elected as cluster manager?

In [2]:
! curl -u admin:admin -k -X GET "https://localhost:9200/_cat/nodes?v"

ip         heap.percent ram.percent cpu load_1m load_5m load_15m node.role node.roles                                        cluster_manager name
172.18.0.2           54          40   1    0.31    0.27     0.28 dimr      cluster_manager,data,ingest,remote_cluster_client -               opensearch-node3
172.18.0.5           66          40   1    0.31    0.27     0.28 dimr      cluster_manager,data,ingest,remote_cluster_client -               opensearch-node2
172.18.0.3           35          40   2    0.31    0.27     0.28 dimr      cluster_manager,data,ingest,remote_cluster_client *               opensearch-node1


A1: opensearch-node1

In [3]:
! docker stop opensearch-node1

opensearch-node1


In [7]:
# ip         heap.percent ram.percent cpu load_1m load_5m load_15m node.role node.roles                                        cluster_manager name
# 172.18.0.5           24          30   1    0.25    0.25     0.27 dimr      cluster_manager,data,ingest,remote_cluster_client -               opensearch-node2
# 172.18.0.2           15          30   1    0.25    0.25     0.27 dimr      cluster_manager,data,ingest,remote_cluster_client *               opensearch-node3


Q2: After stopping the previous cluster manager, which node was elected the new cluster manager?

A2: opensearch-node3

In [11]:
# opensearch-node3       | [2023-05-13T16:44:40,757][INFO ][o.o.c.c.Coordinator      ] [opensearch-node3] cluster-manager node [{opensearch-node1}{GJkgkvlTTGmkXhDgvgIGCw}{2yMOv11RSz6YQKWd8DlR1w}{172.18.0.3}{172.18.0.3:9300}{dimr}{shard_indexing_pressure_enabled=true}] failed, restarting discovery
# [2023-05-13T16:44:40,929][INFO ][o.o.c.s.MasterService    ] [opensearch-node3] elected-as-cluster-manager ([2] nodes joined)[{opensearch-node3}{6eiG0tSUTQSn7hSKGTfvDg}{W2-EHRvARZGlqZ23fgfUEg}{172.18.0.2}{172.18.0.2:9300}{dimr}{shard_indexing_pressure_enabled=true} elect leader, {opensearch-node2}{an19axTsRReTRjcVO1VhZA}{rsye-Pj_SpmX8HFuu2Pj6w}{172.18.0.5}{172.18.0.5:9300}{dimr}{shard_indexing_pressure_enabled=true} elect leader, _BECOME_CLUSTER_MANAGER_TASK_, _FINISH_ELECTION_], term: 6, version: 72, delta: cluster-manager node changed {previous [], current [{opensearch-node3}{6eiG0tSUTQSn7hSKGTfvDg}{W2-EHRvARZGlqZ23fgfUEg}{172.18.0.2}{172.18.0.2:9300}{dimr}{shard_indexing_pressure_enabled=true}]}

In [12]:
! docker start opensearch-node1

opensearch-node1


Q3: Did the cluster manager node change again? (was a different node elected as cluster manager when you started the node back up?)

In [13]:
! curl -u admin:admin -k -X GET "https://localhost:9200/_cat/nodes?v"

ip         heap.percent ram.percent cpu load_1m load_5m load_15m node.role node.roles                                        cluster_manager name
172.18.0.2           25          40   1    0.32    0.25     0.21 dimr      cluster_manager,data,ingest,remote_cluster_client *               opensearch-node3
172.18.0.5           30          40   1    0.32    0.25     0.21 dimr      cluster_manager,data,ingest,remote_cluster_client -               opensearch-node2
172.18.0.3           31          40   1    0.32    0.25     0.21 dimr      cluster_manager,data,ingest,remote_cluster_client -               opensearch-node1


A3: No, cluster manager node did not change.

### Level 2

In [19]:
import os
os.environ["WEEK3_PATH"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/search_engineering/week3"
os.environ["BBUY_DATA"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products"


In [21]:
! curl -k -X PUT -u admin:admin "https://localhost:9200/bbuy_products" -H 'Content-Type: application/json' -d @$WEEK3_PATH/bbuy_products.json

{"acknowledged":true,"shards_acknowledged":true,"index":"bbuy_products"}

In [22]:
!python index.py -s $BBUY_DATA -w 8 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 8 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 3.5216314651 minutes.  Total accumulated time spent in `bulk` indexing: 9.139541834033327 minutes


In [23]:
! curl -u admin:admin -k -X GET "https://localhost:9200/_cat/shards/bbuy_products?v&s=shard,prirep"

index         shard prirep state    docs  store ip         node
bbuy_products 0     p      STARTED 84880 90.1mb 172.18.0.3 opensearch-node1
bbuy_products 0     r      STARTED 84880 94.5mb 172.18.0.2 opensearch-node3
bbuy_products 0     r      STARTED 84880 94.7mb 172.18.0.5 opensearch-node2
bbuy_products 1     p      STARTED 85376 92.8mb 172.18.0.2 opensearch-node3
bbuy_products 1     r      STARTED 85376 88.6mb 172.18.0.3 opensearch-node1
bbuy_products 1     r      STARTED 85376 94.9mb 172.18.0.5 opensearch-node2
bbuy_products 2     p      STARTED 84821 93.7mb 172.18.0.5 opensearch-node2
bbuy_products 2     r      STARTED 84821 93.9mb 172.18.0.3 opensearch-node1
bbuy_products 2     r      STARTED 84821 94.4mb 172.18.0.2 opensearch-node3


In [25]:
! curl -u admin:admin -k -X DELETE "https://localhost:9200/bbuy_products/"

{"acknowledged":true}

In [26]:
! curl -k -X PUT -u admin:admin "https://localhost:9200/bbuy_products" -H 'Content-Type: application/json' -d @$WEEK3_PATH/bbuy_products.json

{"acknowledged":true,"shards_acknowledged":true,"index":"bbuy_products"}

In [27]:
!python index.py -s $BBUY_DATA -w 8 -b 500 -m 1000

INFO:Indexing /Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/product_data/products to bbuy_products with 8 workers, refresh_interval of -1 to host localhost with a maximum number of docs sent per file per worker of 1000 and 500 per batch.
INFO:Done. 255077 were indexed in 2.4895814310499995 minutes.  Total accumulated time spent in `bulk` indexing: 3.1895598371166676 minutes


In [28]:
! curl -u admin:admin -k -X GET "https://localhost:9200/_cat/shards/bbuy_products?v&s=shard,prirep"

index         shard prirep state    docs  store ip         node
bbuy_products 0     p      STARTED 84880 90.8mb 172.18.0.3 opensearch-node1
bbuy_products 1     p      STARTED 85376 92.6mb 172.18.0.2 opensearch-node3
bbuy_products 2     p      STARTED 84821 89.6mb 172.18.0.5 opensearch-node2


Q4: How much faster was it to index the dataset with 0 replicas versus the previous time with 2 replica shards?

A4: 2.48 minutes vs 3.58 minutes

Q5: Why was it faster?
A5: Replication is done sequentialy after indexing the document in primary shard. Succesfull indexing of one document requires replication in all replicas to be finished. Thus dropping replicas cuts the indexing time.

In [29]:
! curl -k -XPUT -u admin:admin 'https://localhost:9200/bbuy_products/_settings' -H 'Content-Type: application/json' -d '{ "index": { "number_of_replicas": 2} }'

{"acknowledged":true}

```
[2023-05-13T19:33:50,603][INFO ][o.o.c.m.MetadataUpdateSettingsService] [opensearch-node3] updating number_of_replicas to [2] for indices [bbuy_products]
[2023-05-13T19:34:00,293][INFO ][o.o.c.r.a.AllocationService] [opensearch-node3] Cluster health status changed from [YELLOW] to [GREEN] (reason: [shards started [[bbuy_products][1]]]).
```

Q6: How long did it take to create the new replica shards?  This will be the difference in time between those two log messages.

A6: 10 seconds

Q7: Those two messages were both logged by the cluster_manager.  Why do you think the cluster manager is the node that logs these actions (versus non-manager nodes)?

A7: Cluster manager is the source of truth and is co-ordinating all the work related to shards/replicas placements.

In [30]:
! curl -u admin:admin -k -X GET "https://localhost:9200/_cat/shards/bbuy_products?v&s=shard,prirep"

index         shard prirep state    docs  store ip         node
bbuy_products 0     p      STARTED 84880 90.8mb 172.18.0.3 opensearch-node1
bbuy_products 0     r      STARTED 84880 90.8mb 172.18.0.2 opensearch-node3
bbuy_products 0     r      STARTED 84880 90.8mb 172.18.0.5 opensearch-node2
bbuy_products 1     p      STARTED 85376 92.6mb 172.18.0.2 opensearch-node3
bbuy_products 1     r      STARTED 85376 92.6mb 172.18.0.3 opensearch-node1
bbuy_products 1     r      STARTED 85376 92.6mb 172.18.0.5 opensearch-node2
bbuy_products 2     p      STARTED 84821 89.6mb 172.18.0.5 opensearch-node2
bbuy_products 2     r      STARTED 84821 89.6mb 172.18.0.3 opensearch-node1
bbuy_products 2     r      STARTED 84821 89.6mb 172.18.0.2 opensearch-node3


### Level 3

In [31]:
os.environ["BBUY_QUERIES"] = "/Users/krzysztof.mioduszews/Development/Courses/corise-search-eng/datasets/"

In [ ]:
!python query.py -q $BBUY_QUERIES/train.csv -w 4 -m 25000

![QPS](qps.png)

Whole system almost 1k QPS. Week2 best configuration (4 CPU, 4 GB heap size) was around 300 QPS.